In [ ]:
import pandas as pd
import os
import random 
import numpy as np
random.seed(42)

import sys 
sys.path.append('../src/utils/')
from utils import score_minimality, compute_dist

def create_dir_and_export(export_df, path, name, split):
    path= path + '/' + name + '/'

    if not os.path.exists(path):
        os.makedirs(path)

    export_df.to_csv(path + split + '.tsv', sep='\t', index = False)

def flip_label(x):
    if x == 'hate':
        return 'nothate'
    elif x == 'nothate':
        return 'hate'
    else:
        raise ValueError

def remove_label(x):
    if x.startswith('hate:'):
        return x[len('hate:'):].strip()
    elif x.startswith('nothate:'):
        return x[len('nothate:'):].strip()
    else:
        return x.strip()

In [ ]:
#DATA_NAME = 'gpt4-20240318'
#DATA_NAME = 'mistral-20240118'
#DATA_NAME = 'counterfactually-augmented-data'


for LLM in ['counterfactually-augmented-data', 'gpt3.5-20240313', 'gpt4-20240318', 'mistral-20240118']:
    DATA_NAME = LLM
    SPLITS = [ 'test']
    #SPLITS = ['train']

    for split_name in SPLITS:
        #split_name = 'train'
        path_raw = '../llms-raw/{}/speech/{}.csv'.format(DATA_NAME, split_name)
        df = pd.read_csv(path_raw)
            
        df.dropna(inplace=True)
        df = df[df['contrast label'] != df['original label']]
        df = df[df['contrast text'] != df['original_text']]


        df['contrast text'] = df['contrast text'].map(lambda x: x.split('\n')[0].strip())

        df['contrast text'].replace('', np.nan, inplace=True)
            #df = df[df['llm_contrast'] != '']

        df.dropna(inplace=True)
        assert not (df.isnull().values.any())



        if 'index' in df.columns:
            df.drop(columns=['index'], inplace=True)
        print(split_name)
        print('### combined DATA ###')
        print(df.head())
        print('####################')

        for i, row in df.iterrows():
            if row['original label'] == row['contrast label']:
                print(i)
                print(row['contrast text'])
                print(row['original_text'])
                raise ValueError

        df_original = df[['original_text', 'original label']]
        df_new = df[[ 'contrast text', 'original label']].copy(deep=True)
        # Flip original sentiment
        df_new['original label'] = df_new['original label'].apply(flip_label)
        # Rename column to match original 
        df_new.rename(columns={'contrast text' : 'Text', 'original label' : 'Label'}, inplace=True)
        df_original.rename(columns={'original_text' : 'Text', 'original label' : 'Label'}, inplace=True)

        for tmp_df in [df_new, df_original]:
            tmp_df['Text'] = tmp_df.apply(lambda x: remove_label(x['Text']), axis=1) 

        print('### NEW DATA ###')
        print(df_new.head())
        print('####################')

        assert (len(df_original) == len(df_new))
        for i, j in zip(df_original.iterrows(), df_new.iterrows()):
            assert(i[1]['Label'] != j[1]['Label'])
            if i[1]['Label'] == 'hate':
                assert (j[1]['Label'] == 'nothate')
            elif i[1]['Label'] == 'nothate':
                assert (j[1]['Label'] == 'hate')


        assert not (df_original.isnull().values.any())
        assert not (df_new.isnull().values.any())
        dist_column = compute_dist(df_original['Text'].values, df_new['Text'].values)
        path = f'../llms-ppl-preds/{LLM}/speech/new/{split_name}.tsv'
        df_export = pd.read_csv(path, sep='\t')

        assert len(dist_column) == len(df_export)

        df_export['dist'] = dist_column
        df_export.to_csv(path, index=False, sep='\t')



